In [2]:
!pip install geopy
!pip install folium

     |████████████████████████████████| 94 kB 5.1 MB/s  eta 0:00:01


In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('STATUS: Libraries imported.')
# <SPAN STYLE="text-decoration:underline">  <span style="color:blue">  1.0	Introduction Section </span> 

STATUS: Libraries imported.


In [4]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

Data downloaded!


In [5]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [6]:
neighborhoods_data = newyork_data['features']

In [7]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [8]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [9]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


### <SPAN STYLE="">  <span style="color:blue">  Now lets make a map of New York Cityn </span> 

In [11]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [12]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

### <SPAN STYLE="">  <span style="color:blue">  More specifically, my client wants to look at Manhattan </span> 

In [13]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [14]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [15]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
### <SPAN STYLE="">  <span style="color:blue">  More specifically, my client wants to look at Manhattan </span> map_manhattan

### <SPAN STYLE="">  <span style="color:blue">  Using Foursquare API to learn more about each neighborhood </span> 


In [16]:
CLIENT_ID = 'ZWLCLXKRKKBH1AXN2EMN0TLRRVCPDINO1CYW4AVA5VUFANK3' # your Foursquare ID
CLIENT_SECRET = '2OKBDY3EH23YLKNIS1UEDB4BHBGBLB2SP4EPOABWQ0LCJF5X' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZWLCLXKRKKBH1AXN2EMN0TLRRVCPDINO1CYW4AVA5VUFANK3
CLIENT_SECRET:2OKBDY3EH23YLKNIS1UEDB4BHBGBLB2SP4EPOABWQ0LCJF5X


### <SPAN STYLE="">  <span style="color:blue">  Now we want to look around an affordable neighborhood like Yorkville </span> 

In [17]:
manhattan_data.loc[9, 'Neighborhood']

'Yorkville'

In [18]:
neighborhood_latitude = manhattan_data.loc[9, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[9, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[9, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Yorkville are 40.775929849884875, -73.94711784471826.


### <SPAN STYLE="">  <span style="color:blue">  Next lets find nearby venues within Yorkville </span> 

In [19]:
# type your answer here
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=ZWLCLXKRKKBH1AXN2EMN0TLRRVCPDINO1CYW4AVA5VUFANK3&client_secret=2OKBDY3EH23YLKNIS1UEDB4BHBGBLB2SP4EPOABWQ0LCJF5X&v=20180605&ll=40.775929849884875,-73.94711784471826&radius=500&limit=100'

In [20]:
results = requests.get(url).json()

In [21]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [22]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Bagel Bob's on York,Bagel Shop,40.776459,-73.946972
1,EVF Performance,Gym,40.775625,-73.947402
2,Yorkshire Wines & Spirits,Wine Shop,40.776319,-73.949786
3,Park East Wines & Spirits,Liquor Store,40.776715,-73.946663
4,Mansion Restaurant,Diner,40.775962,-73.946777
5,Alex's MVP Comics & Cards,Hobby Shop,40.774434,-73.948286
6,Carl Schurz Park,Park,40.775118,-73.943763
7,Peng's Noodle Folk,Asian Restaurant,40.777258,-73.949110
8,Le Grand Triage: Wine & Whiskey,Wine Shop,40.777198,-73.949167
9,Stella & Fly,Coffee Shop,40.778464,-73.948282


In [23]:

# create map of Yorkville using latitude and longitude values
map_yorkville = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, label in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['categories']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_yorkville)  
    
map_yorkville

### <SPAN STYLE="">  <span style="color:blue">  Expand this to more affordable neighborhoods </span> 

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
         # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [26]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

### <SPAN STYLE="">  <span style="color:blue">  Finding Affordable neighborhood offers the best combinations of venues </span> 

In [27]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()

In [28]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
            venue  freq
0     Coffee Shop  0.07
1            Park  0.05
2  Clothing Store  0.05
3           Hotel  0.05
4             Gym  0.05


----Carnegie Hill----
         venue  freq
0  Coffee Shop  0.08
1         Café  0.05
2  Yoga Studio  0.04
3  Pizza Place  0.04
4    Bookstore  0.04


----Central Harlem----
                venue  freq
0  African Restaurant  0.07
1  Chinese Restaurant  0.05
2      Cosmetics Shop  0.05
3  Seafood Restaurant  0.05
4   French Restaurant  0.05


----Chelsea----
                 venue  freq
0               Bakery  0.05
1          Coffee Shop  0.05
2  American Restaurant  0.04
3          Art Gallery  0.04
4                Hotel  0.03


----Chinatown----
                 venue  freq
0               Bakery  0.08
1   Chinese Restaurant  0.07
2         Cocktail Bar  0.04
3    Hotpot Restaurant  0.04
4  American Restaurant  0.04


----Civic Center----
                  venue  freq
0           Coffee Shop  0.08
1          Cocktai

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

### East Harlem

In [31]:
neighborhoods_venues_sorted.loc[7,:]

Neighborhood                            East Harlem
1st Most Common Venue            Mexican Restaurant
2nd Most Common Venue               Thai Restaurant
3rd Most Common Venue                        Bakery
4th Most Common Venue                 Deli / Bodega
5th Most Common Venue     Latin American Restaurant
6th Most Common Venue                Sandwich Place
7th Most Common Venue                      Beer Bar
8th Most Common Venue                 Grocery Store
9th Most Common Venue                  Liquor Store
10th Most Common Venue                   Taco Place
Name: 7, dtype: object

### Hamilton Heights

In [32]:
neighborhoods_venues_sorted.loc[13,:]

Neighborhood                       Hamilton Heights
1st Most Common Venue                   Pizza Place
2nd Most Common Venue                   Coffee Shop
3rd Most Common Venue                          Café
4th Most Common Venue            Mexican Restaurant
5th Most Common Venue                 Deli / Bodega
6th Most Common Venue                   Yoga Studio
7th Most Common Venue     Latin American Restaurant
8th Most Common Venue                  Liquor Store
9th Most Common Venue                          Park
10th Most Common Venue                 Cocktail Bar
Name: 13, dtype: object

### Inwood

In [33]:
neighborhoods_venues_sorted.loc[15,:]

Neighborhood                            Inwood
1st Most Common Venue       Mexican Restaurant
2nd Most Common Venue                     Café
3rd Most Common Venue                   Lounge
4th Most Common Venue               Restaurant
5th Most Common Venue                     Park
6th Most Common Venue     Caribbean Restaurant
7th Most Common Venue                   Bakery
8th Most Common Venue       Chinese Restaurant
9th Most Common Venue              Pizza Place
10th Most Common Venue                Wine Bar
Name: 15, dtype: object

### Murray Hill

In [34]:
neighborhoods_venues_sorted.loc[26,:]

Neighborhood                       Murray Hill
1st Most Common Venue              Coffee Shop
2nd Most Common Venue                    Hotel
3rd Most Common Venue                      Bar
4th Most Common Venue           Sandwich Place
5th Most Common Venue      Japanese Restaurant
6th Most Common Venue     Gym / Fitness Center
7th Most Common Venue             Burger Joint
8th Most Common Venue                      Pub
9th Most Common Venue                Juice Bar
10th Most Common Venue        Sushi Restaurant
Name: 26, dtype: object

### Washington Heights

In [35]:
neighborhoods_venues_sorted.loc[37,:]

Neighborhood                     Washington Heights
1st Most Common Venue                          Café
2nd Most Common Venue                 Deli / Bodega
3rd Most Common Venue                        Bakery
4th Most Common Venue             Mobile Phone Shop
5th Most Common Venue                          Bank
6th Most Common Venue     Latin American Restaurant
7th Most Common Venue       New American Restaurant
8th Most Common Venue                          Park
9th Most Common Venue                   Pizza Place
10th Most Common Venue                          Gym
Name: 37, dtype: object

### Yorkville

In [36]:
neighborhoods_venues_sorted.loc[39,:]

Neighborhood                        Yorkville
1st Most Common Venue      Italian Restaurant
2nd Most Common Venue                     Gym
3rd Most Common Venue             Coffee Shop
4th Most Common Venue        Sushi Restaurant
5th Most Common Venue           Deli / Bodega
6th Most Common Venue                     Bar
7th Most Common Venue     Japanese Restaurant
8th Most Common Venue      Mexican Restaurant
9th Most Common Venue               Wine Shop
10th Most Common Venue             Bagel Shop
Name: 39, dtype: object

#  Visualize these neighbourhoods

### Murray Hill

In [37]:
manhattan_data.loc[16, 'Neighborhood']

'Murray Hill'

In [38]:
neighborhood_latitude = manhattan_data.loc[16, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[16, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[16, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Murray Hill are 40.748303077252174, -73.97833207924127.


In [39]:
# type your answer here
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=ZWLCLXKRKKBH1AXN2EMN0TLRRVCPDINO1CYW4AVA5VUFANK3&client_secret=2OKBDY3EH23YLKNIS1UEDB4BHBGBLB2SP4EPOABWQ0LCJF5X&v=20180605&ll=40.748303077252174,-73.97833207924127&radius=500&limit=100'

In [40]:
results = requests.get(url).json()

In [41]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [42]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Ippodo Tea Co.,Tea Room,40.749757,-73.977733
1,Kajitsu,Japanese Restaurant,40.749763,-73.977688
2,Perk Kafe,Coffee Shop,40.747768,-73.977363
3,Sons of Thunder,Hawaiian Restaurant,40.747970,-73.975751
4,Shake Shack,Burger Joint,40.749666,-73.975719


In [43]:
# create map of Murray Hill using latitude and longitude values
map_murrayhill = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['categories']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_murrayhill)  
    
map_murrayhill

### East Harlem

In [44]:
manhattan_data.loc[7, 'Neighborhood']

'East Harlem'

In [45]:
neighborhood_latitude = manhattan_data.loc[7, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[7, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[7, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of East Harlem are 40.79224946663033, -73.94418223148524.


In [46]:
# type your answer here
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=ZWLCLXKRKKBH1AXN2EMN0TLRRVCPDINO1CYW4AVA5VUFANK3&client_secret=2OKBDY3EH23YLKNIS1UEDB4BHBGBLB2SP4EPOABWQ0LCJF5X&v=20180605&ll=40.79224946663033,-73.94418223148524&radius=500&limit=100'

In [47]:
results = requests.get(url).json()

In [48]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [49]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,East Harlem Bottling Co.,Beer Bar,40.793024,-73.945727
1,El Chevere Cuchifritos,Latin American Restaurant,40.793994,-73.942971
2,Cascolate Latin Bistro,Mexican Restaurant,40.792787,-73.940830
3,Malii,Thai Restaurant,40.789525,-73.943232
4,Savoy Bakery,Bakery,40.794456,-73.943526


In [50]:
# create map of East Harlem using latitude and longitude values
map_eastharlem = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['categories']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_eastharlem)  
    
map_eastharlem

### Hamilton Heights

In [51]:
manhattan_data.loc[4, 'Neighborhood']

'Hamilton Heights'

In [52]:
neighborhood_latitude = manhattan_data.loc[4, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[4, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[4, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Hamilton Heights are 40.823604284811935, -73.94968791883366.


In [53]:
# type your answer here
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=ZWLCLXKRKKBH1AXN2EMN0TLRRVCPDINO1CYW4AVA5VUFANK3&client_secret=2OKBDY3EH23YLKNIS1UEDB4BHBGBLB2SP4EPOABWQ0LCJF5X&v=20180605&ll=40.823604284811935,-73.94968791883366&radius=500&limit=100'

In [54]:
results = requests.get(url).json()

In [55]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [56]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,ROKC,Cocktail Bar,40.823654,-73.952506
1,The Grange Bar & Eatery,Cocktail Bar,40.822554,-73.949532
2,Cafe One,Café,40.822139,-73.949853
3,Fumo,Italian Restaurant,40.821412,-73.950499
4,Bikram Yoga Harlem,Yoga Studio,40.825672,-73.948887


In [62]:
# create map of Hamilton Heights using latitude and longitude values
map_hamheights = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['categories']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_hamheights)  
    
map_hamheights

### Inwood

In [63]:
manhattan_data.loc[3, 'Neighborhood']

'Inwood'

In [64]:
neighborhood_latitude = manhattan_data.loc[3, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[3, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[3, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Inwood are 40.86768396449915, -73.92121042203897.


In [65]:
# type your answer here
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=ZWLCLXKRKKBH1AXN2EMN0TLRRVCPDINO1CYW4AVA5VUFANK3&client_secret=2OKBDY3EH23YLKNIS1UEDB4BHBGBLB2SP4EPOABWQ0LCJF5X&v=20180605&ll=40.86768396449915,-73.92121042203897&radius=500&limit=100'

In [66]:
results = requests.get(url).json()

In [67]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [68]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Inwood Farmers Market,Farmers Market,40.869062,-73.920560
1,PJ Wine,Wine Shop,40.867251,-73.922349
2,ChocNYC,Bakery,40.868396,-73.918742
3,Inwood Gourmet,Deli / Bodega,40.868594,-73.921361
4,Inwood Local,Wine Bar,40.868032,-73.920581


In [73]:
# create map of Inwood using latitude and longitude values
map_inwood = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['categories']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_inwood)  
    
map_inwood

### Washington Heights

In [74]:
manhattan_data.loc[2, 'Neighborhood']

'Washington Heights'

In [75]:
neighborhood_latitude = manhattan_data.loc[2, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[2, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[2, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Washington Heights are 40.85190252555305, -73.93690027985234.


In [76]:
# type your answer here
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=ZWLCLXKRKKBH1AXN2EMN0TLRRVCPDINO1CYW4AVA5VUFANK3&client_secret=2OKBDY3EH23YLKNIS1UEDB4BHBGBLB2SP4EPOABWQ0LCJF5X&v=20180605&ll=40.85190252555305,-73.93690027985234&radius=500&limit=100'

In [77]:
results = requests.get(url).json()

In [78]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [79]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,The Uptown Garrison,Restaurant,40.851255,-73.939473
1,Green Juice Cafe,Café,40.851898,-73.934827
2,Jin's Superette,Deli / Bodega,40.850989,-73.938514
3,Le Chéile,Café,40.851459,-73.939989
4,Saggio Restaurant,Italian Restaurant,40.851423,-73.939761


In [80]:
# create map of Washington Heights using latitude and longitude values
map_washheights = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['categories']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_washheights)  
    
map_washheights

### ** Project Completed Successfully **